In [1]:
using Random
using DataFrames
using CSV

In [15]:
function build_lattice_df(Lx,Ly)
    
    N = Lx*Ly
    sites = collect(1:N)
    dirs = ['x','y']
    interactions = DataFrame(["$site,$dir" => [] for site in sites for dir in dirs])    
    for site in 1:N
        up = mod(site-1-Lx,N)
        down = mod(site-1+Lx,N)
        right = Int(((site-1)-((site-1)%Lx)) + mod(((site-1)%Lx)+1,Lx))
        left = Int(((site-1)-((site-1)%Lx)) + mod(((site-1)%Lx)-1,Lx))
        push!(interactions[!,"$site,x"],[left+1,right+1])
        push!(interactions[!,"$site,y"],[up+1,down+1])
    end
    
    return interactions
end

build_lattice_df (generic function with 1 method)

In [16]:
function rows_and_columns(Lx,Ly)
    
    N =Lx*Ly
    rows = DataFrame(["row$row" => [] for row in 1:Ly])    
    cols = DataFrame(["col$col" => [] for col in 1:Lx])
    for site in 1:N
        row = Int((((site-1) - ((site-1)%Lx))/Ly)+1)
        col = Int(((site-1) % Lx)+1)
        push!(rows[!,"row$row"],site)
        push!(cols[!,"col$col"],site)
    end
    
    return rows, cols
end

rows_and_columns (generic function with 1 method)

In [17]:
function Calculate_XY_Energy(config,interactions)
    
    N = length(config)
    totalenergy = 0
    
    for site in (1:N)
        neighbors_x = interactions[!,"$site,x"][1,1]
        neighbors_y = interactions[!,"$site,y"][1,1]
        energy_contributions_x = 0.5*-1*cos.(config[site].-config[neighbors_x])
        energy_contributions_y = 0.5*-1*cos.(config[site].-config[neighbors_y])
        totalenergy += sum(energy_contributions_x) + sum(energy_contributions_y)
    end
    
    return totalenergy

end

Calculate_XY_Energy (generic function with 1 method)

In [18]:
test1 = ones(16)*0.5;
interactions = build_lattice_df(4,4);
test_energy = Calculate_XY_Energy(test1,interactions)

-32.0

In [19]:
function XY_update_function(config,energy,interactions,beta,energy_func)
    
    random_angles =rand(Float64,length(config))*2*pi
    new_config = mod2pi.(copy(config).+random_angles)
    new_energy = energy_func(new_config,interactions)
    deltaE = new_energy-energy
    exp_beta_deltaE = exp(-1*beta*deltaE)
    rand_value = rand(Float64,1)[1]
    
    if exp_beta_deltaE > rand_value
        config = new_config
    else
        deltaE = 0.
    end
    
    return config, deltaE
end

XY_update_function (generic function with 1 method)

In [20]:
test2 = rand(Float64,16)*2*pi
test_energy2 = Calculate_XY_Energy(test2,interactions)
XY_update_function(test2,test_energy2,interactions,1,Calculate_XY_Energy)

([0.9631328453442337, 1.1969991509486162, 0.5765176655244939, 0.3300227758897261, 0.5503292209215372, 0.5953070065158568, 2.156700516561668, 0.6956246436390606, 0.06110798333552896, 1.5577150078960698, 2.482527617506036, 2.3567100963915903, 0.995350677392233, 1.0185980670732253, 4.182837503925386, 1.5346694909665168], 0.0)

In [21]:
function spin_stiffness(beta,config,interactions,rows,cols)
    
    N = length(config)
    L = Int(sqrt(N)) # obv for a square lattice only
    
    avg_ps_over_rows = 0
    for row in 1:L
        sites_in_row = rows[!,"row$row"]
        cos_contributions_x = 0
        sin_contributions_x = 0
        for site in sites_in_row
            neighbors_x = interactions[!,"$site,x"][1,1]
            cos_contributions_x += sum(0.5*-1*cos.(config[site].-config[neighbors_x]))
            sin_contributions_x += sum(0.5*-1*sin.(config[site].-config[neighbors_x]))
        end
        avg_ps_over_rows += ((1/L) * (1/N) * cos_contributions_x) - ((1/L) * (beta/N) * sin_contributions_x^2)
    end
    
    avg_ps_over_cols = 0
    for col in 1:L
        sites_in_cols = cols[!,"col$col"]
        cos_contributions_y = 0
        sin_contributions_y = 0
        for site in sites_in_cols
            neighbors_y = interactions[!,"$site,y"][1,1]
            cos_contributions_y += sum(0.5*-1*cos.(config[site].-config[neighbors_y]))
            sin_contributions_y += sum(0.5*-1*sin.(config[site].-config[neighbors_y]))
        end
        avg_ps_over_cols += ((1/L) * (1/N) * cos_contributions_y) - ((1/L) * (beta/N) * sin_contributions_y^2)
    end
    
    spin_stiffness = (avg_ps_over_rows+avg_ps_over_cols)/2
            
    return spin_stiffness, avg_ps_over_rows, avg_ps_over_cols
end

spin_stiffness (generic function with 1 method)

In [22]:
rows,cols = rows_and_columns(4,4)
spin_stiffness(1,test1,interactions,rows,cols)

(-0.25, -0.25, -0.25)

In [23]:
chk = [-1,1,-1,1,1,-1,1,-1,-1,1,-1,1,1,-1,1,-1]
spin_stiffness(1,chk,interactions,rows,cols)

(0.1040367091367856, 0.1040367091367856, 0.1040367091367856)

In [32]:
function MonteCarlo_Ising(beta,Lx,Ly,warmup_steps,steps,steps_per_step,build_lattice_function,rows_and_cols_function,update_function,energy_function,spin_stiffness_function)
    
    # system information
    N = Lx*Ly
    interactions = build_lattice_function(Lx,Ly)
    rows,cols = rows_and_cols_function(Lx,Ly)
    initial_config = rand(Float64,N)*2*pi
    
    # initialize dataframes
    maindf = DataFrame(:avenergy => Float64[],:avenergy2 => Float64[],:pss => Float64[],:avpss => Float64[])
    
    # warmup steps - track Es and Ms but not in averages
    config = initial_config
    energy = energy_function(config,interactions)
    for step in (1:warmup_steps)
        for inner_steps in (1:steps_per_step)
            new_config, deltaE = update_function(config,energy,interactions,beta,energy_function)
            config = copy(new_config)
            new_energy = energy+deltaE
            energy = new_energy
        end
    end
    
    # begin MC steps and observable tracking
    sumE = 0
    sumE2 = 0
    sumps = 0
    println("Warmup steps over!")
    for step in (1:steps)
        for inner_step in (1:steps_per_step)
            new_config,deltaE = update_function(config,energy,interactions,beta,energy_function)
            config = copy(new_config)
            new_energy = energy+deltaE
            energy = new_energy
        end
        sumE = sumE+energy
        sumE2 = sumE2+energy^2
        ps,_,_ = spin_stiffness_function(beta,config,interactions,rows,cols)
        sumps += ps
        push!(maindf, (sumE/step,sumE2/step,ps,sumps/step))
    end
        
    return maindf
end

MonteCarlo_Ising (generic function with 2 methods)

In [33]:
MonteCarlo_Ising(1,4,4,1000,1000,10,build_lattice_df,rows_and_columns,XY_update_function,Calculate_XY_Energy,spin_stiffness)


Warmup steps over!


Row,avenergy,avenergy2,pss,avpss
,Float64,Float64,Float64,Float64
1,-15.8872,252.404,-0.124119,-0.124119
2,-15.8872,252.404,-0.124119,-0.124119
3,-15.8872,252.404,-0.124119,-0.124119
4,-15.8872,252.404,-0.124119,-0.124119
5,-15.8872,252.404,-0.124119,-0.124119
6,-15.8872,252.404,-0.124119,-0.124119
7,-15.8872,252.404,-0.124119,-0.124119
8,-15.8872,252.404,-0.124119,-0.124119
9,-15.8872,252.404,-0.124119,-0.124119
